In [ ]:
'''
Make sure you that when you export participants list from Zoom
that you check both "Export with meeting data" and "Show unique users".

See here:
https://www.eduhk.hk/ocio/content/faq-how-retrieve-attendance-list-zoom-meeting
'''

In [ ]:
"""
This script processes attendance data by comparing a list of expected participants against a list of actual participants.
It loads two CSV files: one containing the expected participants with their official and informal names, and the other containing the actual participants.
The script then replaces the informal names with the official names.
Next, it determines who attended the event, who did not attend, and who is unknown.
Finally, it creates a DataFrame with the attendance information, including the names of the attendees and their attendance status (Present, Absent, or Unknown).
"""

In [ ]:
import pandas as pd

# Load expected participants
expected_participants = pd.read_csv('expected_participants.csv')
expected_participants_list = set(expected_participants['Official Name'])
expected_participants_dict = dict(zip(expected_participants['Name (Original Name)'], 
                                      expected_participants['Official Name']))

# Load actual participants, skipping the first two rows
df = pd.read_csv('participants_1234567890.csv', skiprows=2, header=1)[['Name (Original Name)']]

# Replace the informal name with the official name or keep the informal name if not in the dictionary
actual_participants_list = df['Name (Original Name)'].apply(lambda x: expected_participants_dict.get(x, x)).tolist()

# Determine who attended, who did not attend, and who is unknown
present = set(actual_participants_list).intersection(expected_participants_list)
absent = set(expected_participants_list).difference(actual_participants_list)
unknown = [name for name in actual_participants_list if name not in expected_participants_list]

# Create a dataframe for attendance status
attendance_df = (pd.concat([pd.DataFrame(present, columns=['Name']).assign(Status='Present'),
                            pd.DataFrame(absent, columns=['Name']).assign(Status='Absent')])
                 .sort_values('Name'))

# Add unknown attendees to the dataframe
unknown_df = pd.DataFrame(unknown, columns=['Name']).assign(Status='Present (Unrecognized Name)').reset_index(drop=True)
attendance_df = pd.concat([attendance_df, unknown_df])
attendance_df.to_csv('attendance.csv', index=False)
attendance_df
